# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments           Customer  \
0  Nov  2 2022  9:22AM  249859        10  WT5413 Out   Eywa Pharma Inc.   
1  Nov  2 2022  9:13AM  249855        10  0086138823  ISDIN Corporation   
2  Nov  2 2022  9:13AM  249855        10  0086138818  ISDIN Corporation   
3  Nov  2 2022  9:13AM  249855        10  0086138812  ISDIN Corporation   
4  Nov  2 2022  9:13AM  249855        10  0086138830  ISDIN Corporation   
5  Nov  2 2022  9:13AM  249855        10  0086138843  ISDIN Corporation   
6  Nov  2 2022  9:13AM  249855        10  0086138840  ISDIN Corporation   
7  Nov  2 2022  9:13AM  249855        10  0086138858  ISDIN Corporation   
8  Nov  2 2022  9:13AM  249855        10  0086138859  ISDIN Corporation   
9  Nov  2 2022  9:13AM  249855        10  0086138854  ISDIN Corporation   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
15  249852       Released         10
16  249853       Released          3
17  249854       Released         37
18  249855       Released         13
19  249859       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Executing  Released
id                                 
249852                NaN      10.0
249853                NaN       3.0
249854                NaN      37.0
249855                NaN      13.0
249859                NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Executing  Released
id                                 
249793                1.0       0.0
249794                0.0       1.0
249795                0.0       1.0
249807                5.0      10.0
249828                0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Executing  Released
id                                 
249793                  1         0
249794                  0         1
249795                  0         1
249807                  5        10
249828                  0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Executing  Released
0               249793          1         0
1               249794          0         1
2               249795          0         1
3               249807          5        10
4               249828          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id Executing Released
0               249793         1         
1               249794                  1
2               249795                  1
3               249807         5       10
4               249828                  1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse               Customer
0    Nov  2 2022  9:22AM  249859        10       Eywa Pharma Inc.
1    Nov  2 2022  9:13AM  249855        10      ISDIN Corporation
14   Nov  2 2022  9:09AM  249854        10      ISDIN Corporation
51   Nov  2 2022  9:05AM  249853        19  ACG North America LLC
54   Nov  2 2022  9:01AM  249852        10                Bio-PRF
64   Nov  2 2022  8:51AM  249849        12              Hush Hush
86   Nov  2 2022  8:43AM  249848        19  Eli Lilly and Company
87   Nov  2 2022  8:42AM  249847        10      ISDIN Corporation
100  Nov  2 2022  8:39AM  249846        10      ISDIN Corporation
115  Nov  2 2022  7:54AM  249839        19    AdvaGen Pharma, Ltd

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse               Customer Executing  \
0  Nov  2 2022  9:22AM  249859        10       Eywa Pharma Inc.             
1  Nov  2 2022  9:13AM  249855        10      ISDIN Corporation             
2  Nov  2 2022  9:09AM  249854        10      ISDIN Corporation             
3  Nov  2 2022  9:05AM  249853        19  ACG North America LLC             
4  Nov  2 2022  9:01AM  249852        10                Bio-PRF             
5  Nov  2 2022  8:51AM  249849        12              Hush Hush             
6  Nov  2 2022  8:43AM  249848        19  Eli Lilly and Company             
7  Nov  2 2022  8:42AM  249847        10      ISDIN Corporation             
8  Nov  2 2022  8:39AM  249846        10      ISDIN Corporation         1   
9  Nov  2 2022  7:54AM  249839        19    AdvaGen Pharma, Ltd         1   

  Released  
0        1  
1       13  
2       37  
3        3  
4       10  
5       22  
6        1  
7       13  
8       14  
9        4

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse               Customer Released  \
0  Nov  2 2022  9:22AM  249859        10       Eywa Pharma Inc.        1   
1  Nov  2 2022  9:13AM  249855        10      ISDIN Corporation       13   
2  Nov  2 2022  9:09AM  249854        10      ISDIN Corporation       37   
3  Nov  2 2022  9:05AM  249853        19  ACG North America LLC        3   
4  Nov  2 2022  9:01AM  249852        10                Bio-PRF       10   
5  Nov  2 2022  8:51AM  249849        12              Hush Hush       22   
6  Nov  2 2022  8:43AM  249848        19  Eli Lilly and Company        1   
7  Nov  2 2022  8:42AM  249847        10      ISDIN Corporation       13   
8  Nov  2 2022  8:39AM  249846        10      ISDIN Corporation       14   
9  Nov  2 2022  7:54AM  249839        19    AdvaGen Pharma, Ltd        4   

  Executing  
0            
1            
2            
3            
4            
5            
6            
7            
8         1  
9         1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse               Customer Released  \
0  Nov  2 2022  9:22AM  249859        10       Eywa Pharma Inc.        1   
1  Nov  2 2022  9:13AM  249855        10      ISDIN Corporation       13   
2  Nov  2 2022  9:09AM  249854        10      ISDIN Corporation       37   
3  Nov  2 2022  9:05AM  249853        19  ACG North America LLC        3   
4  Nov  2 2022  9:01AM  249852        10                Bio-PRF       10   

  Executing  
0            
1            
2            
3            
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


Date      id Warehouse               Customer  Released  \
0  Nov  2 2022  9:22AM  249859        10       Eywa Pharma Inc.       1.0   
1  Nov  2 2022  9:13AM  249855        10      ISDIN Corporation      13.0   
2  Nov  2 2022  9:09AM  249854        10      ISDIN Corporation      37.0   
3  Nov  2 2022  9:05AM  249853        19  ACG North America LLC       3.0   
4  Nov  2 2022  9:01AM  249852        10                Bio-PRF      10.0   

   Executing  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Nov  2 2022  9:22AM  249859        10       Eywa Pharma Inc.       1.0   
1  Nov  2 2022  9:13AM  249855        10      ISDIN Corporation      13.0   
2  Nov  2 2022  9:09AM  249854        10      ISDIN Corporation      37.0   
3  Nov  2 2022  9:05AM  249853        19  ACG North America LLC       3.0   
4  Nov  2 2022  9:01AM  249852        10                Bio-PRF      10.0   

   Executing  
0        0.0  
1        0.0  
2        0.0  
3        0.0  
4        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                              
10         1748941      89.0        1.0
12          249849      22.0        0.0
15          249837       2.0        0.0
19         1249178      18.0        7.0
21          749382       2.0        1.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing
0        10  1748941      89.0        1.0
1        12   249849      22.0        0.0
2        15   249837       2.0        0.0
3        19  1249178      18.0        7.0
4        21   749382       2.0        1.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10      89.0        1.0
1        12      22.0        0.0
2        15       2.0        0.0
3        19      18.0        7.0
4        21       2.0        1.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   89.0
1        12  Released   22.0
2        15  Released    2.0
3        19  Released   18.0
4        21  Released    2.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    12   15    19   21
Status                               
Executing   1.0   0.0  0.0   7.0  1.0
Released   89.0  22.0  2.0  18.0  2.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    12   15    19   21
0          Executing   1.0   0.0  0.0   7.0  1.0
1           Released  89.0  22.0  2.0  18.0  2.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    12   15    19   21
0  Executing   1.0   0.0  0.0   7.0  1.0
1   Released  89.0  22.0  2.0  18.0  2.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()